In [152]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd

tickers = ["SPY", "QQQ", "IWM", "EEM"]

In [153]:
data= yf.download(tickers, start="2023-1-1", auto_adjust = False, progress=False)

In [154]:
returns = data['Adj Close'].pct_change().dropna()

In [155]:
window = 20 
ma_20 = data.rolling(window = window).mean()
std_20 = data.rolling(window = window).std()
z_score = (data - ma_20) / std_20

In [156]:
def z_to_signal(z):
    if pd.isna(z):
        return 0
    elif z < -1:
        return 1
    elif z > 1:
        return -1
    else: 
        return 0

signal = z_score.apply(lambda col:col.map(z_to_signal))

In [ ]:
cov_window = 60
rolling_var = returns.rolling(cov_window).var()
inv_var = 1/rolling_var
weights = inv_var.div(inv_var.sum(axis=1), axis = 0)


In [ ]:
common_index = signal.index.intersection(weights.index)
signal_aligned = signal.loc[common_index]
weights_aligned = weights.loc[common_index]

positions = signal_aligned * weights_aligned
print(positions.tail())

In [ ]:
active_signals = signal_aligned.abs()
num_active = active_signals.sum(axis =1)

vanilla_weights = active_signals.div(num_active.replace(0, 1), axis=0)
vanilla_positions = signal_aligned * vanilla_weights
print(vanilla_positions.tail())